In [1]:
# A logistic regression learning algorithm example using TensorFlow library.
# This example is using the MNIST database of handwritten digits 
# (http://yann.lecun.com/exdb/mnist/)

# Author: Aymeric Damien
# Project: https://github.com/aymericdamien/TensorFlow-Examples/

In [2]:
import tensorflow as tf

# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1

# tf Graph Input
x = tf.placeholder(tf.float32, [None, 784]) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, 10]) # 0-9 digits recognition => 10 classes

# Set model weights
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# Construct model
pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

In [5]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print ("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print ("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy for 3000 examples
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print ("Accuracy:", accuracy.eval({x: mnist.test.images[:3000], y: mnist.test.labels[:3000]}))

Epoch: 0001 cost= 1.182138977
Epoch: 0002 cost= 0.664751035
Epoch: 0003 cost= 0.552671993
Epoch: 0004 cost= 0.498528207
Epoch: 0005 cost= 0.465400202
Epoch: 0006 cost= 0.442568785
Epoch: 0007 cost= 0.425419193
Epoch: 0008 cost= 0.412187016
Epoch: 0009 cost= 0.401357577
Epoch: 0010 cost= 0.392349952
Epoch: 0011 cost= 0.384699759
Epoch: 0012 cost= 0.378185341
Epoch: 0013 cost= 0.372374617
Epoch: 0014 cost= 0.367289299
Epoch: 0015 cost= 0.362726761
Epoch: 0016 cost= 0.358611248
Epoch: 0017 cost= 0.354816122
Epoch: 0018 cost= 0.351435964
Epoch: 0019 cost= 0.348324754
Epoch: 0020 cost= 0.345406541
Epoch: 0021 cost= 0.342718719
Epoch: 0022 cost= 0.340235287
Epoch: 0023 cost= 0.337909653
Epoch: 0024 cost= 0.335766982
Epoch: 0025 cost= 0.333657342
Optimization Finished!
Accuracy: 0.889667
